In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from statsmodels.tsa.arima.model import ARIMA

# Load dataset
file_path = "/Users/jaygamage/Downloads/dataf_c.csv"
df = pd.read_csv(file_path)

# Convert date column to datetime format
df['date'] = pd.to_datetime(df['date'])

# Set date as index for time series analysis
df.set_index('date', inplace=True)

# Forecasting for the next 12 months
forecast_steps = 12
forecast_index = pd.date_range(df.index[-1], periods=forecast_steps + 1, freq='M')[1:]

# Define a dictionary to store refined forecasts
refined_forecast_results = {}

# List of columns to forecast
columns_to_forecast = ["gdp", "ncpi", "crudeoil", "traffic_index", "petrol95", "petrol92", "auto_diesel", "super_diesel"]

# Apply ARIMA model for each selected column with optimized parameters
for column in columns_to_forecast:
    model = ARIMA(df[column], order=(7, 1, 7))  # Higher-order ARIMA model for better precision
    model_fit = model.fit()
    forecast_values = model_fit.forecast(steps=forecast_steps)
    refined_forecast_results[column] = forecast_values

# Create a DataFrame with refined forecasts
refined_forecast_df = pd.DataFrame(refined_forecast_results, index=forecast_index)
refined_forecast_df.insert(0, 'Date', forecast_index)

# Define the number of holidays per month in 2025
holidays_per_month = {
    1: 2, 2: 3, 3: 3, 4: 5, 5: 3, 6: 2, 7: 1, 8: 1, 9: 2, 10: 2, 11: 1, 12: 2
}

# Fill in the number of holidays based on the month
refined_forecast_df['num_of_holidays'] = refined_forecast_df['Date'].dt.month.map(holidays_per_month)

# Display the refined forecast
print(refined_forecast_df)
# Save the refined forecast DataFrame to a CSV file
refined_forecast_df.to_csv('/Users/jaygamage/Downloads/ffdata.csv', index=False)

In [152]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
import os

# --- CONFIG ---
TRAIN_PATH = "/Users/jaygamage/Downloads/feature_outputs/train_featured1.csv"
PREDICT_PATH = "/Users/jaygamage/Downloads/feature_outputs/predict_featured1.csv"
TRAIN_OUTPUT = "/Users/jaygamage/Downloads/feature_outputs_new/train_standardized.csv"
PREDICT_OUTPUT = "/Users/jaygamage/Downloads/feature_outputs_new/predict_standardized.csv"

# --- Column Lists ---
train_standardize_cols = [
    'advertising_promotion', 'petrol95', 'petrol92', 'auto_diesel', 'super_diesel',
    'gdp', 'ncpi', 'crudeoil', 'traffic_index', 'sales_quantity_change_pct'
] + [f'sales_quantity_lag_{i}' for i in range(1, 13)] + [
    'sales_quantity_rollmean_3', 'sales_quantity_rollmean_6',
    'sales_quantity_rollmean_9', 'sales_quantity_rollmean_12',
    'sales_quantity_rolling_std_3', 'sales_quantity_rolling_std_6'
]

predict_standardize_cols = [
    "advertising_promotion",
    "gdp", "ncpi", "crudeoil", "traffic_index",
    "petrol95", "petrol92", "auto_diesel", "super_diesel"
]

# --- Load datasets ---
train_df = pd.read_csv(TRAIN_PATH, parse_dates=["date"])
predict_df = pd.read_csv(PREDICT_PATH, parse_dates=["date"])

# --- Store scalers separately ---
train_scalers = {}
predict_scalers = {}

# --- Group-wise Training Standardization ---
for group in train_df["group_code"].unique():
    group_mask = train_df["group_code"] == group
    # Train scaler for full training features
    scaler_train = StandardScaler()
    train_df.loc[group_mask, train_standardize_cols] = scaler_train.fit_transform(
        train_df.loc[group_mask, train_standardize_cols]
    )
    train_scalers[group] = scaler_train

    # Train separate scaler only for prediction feature subset
    scaler_predict = StandardScaler()
    # Use same columns but from training data; rename to match prediction naming
    predict_fit_data = train_df.loc[group_mask, [
        'advertising_promotion', 'gdp', 'ncpi', 'crudeoil', 'traffic_index',
        'petrol95', 'petrol92', 'auto_diesel', 'super_diesel'
    ]].copy()
    predict_fit_data.columns = predict_standardize_cols  # Match prediction column names
    scaler_predict.fit(predict_fit_data)
    predict_scalers[group] = scaler_predict

# --- Ensure output directory exists ---
os.makedirs(os.path.dirname(TRAIN_OUTPUT), exist_ok=True)

# --- Save standardized training data ---
train_df.to_csv(TRAIN_OUTPUT, index=False)
print(f"✅ Group-standardized training data saved to: {TRAIN_OUTPUT}")

# --- Apply matching scaler to prediction data ---
for group in predict_df["group_code"].unique():
    if group in predict_scalers:
        group_mask = predict_df["group_code"] == group
        scaler = predict_scalers[group]
        predict_df.loc[group_mask, predict_standardize_cols] = scaler.transform(
            predict_df.loc[group_mask, predict_standardize_cols]
        )
    else:
        print(f"⚠️ Warning: No scaler found for group {group}. Skipping.")

# --- Save standardized prediction data ---
predict_df.to_csv(PREDICT_OUTPUT, index=False)
print(f"✅ Prediction data standardized using matching scalers saved to: {PREDICT_OUTPUT}")

✅ Group-standardized training data saved to: /Users/jaygamage/Downloads/feature_outputs_new/train_standardized.csv
✅ Prediction data standardized using matching scalers saved to: /Users/jaygamage/Downloads/feature_outputs_new/predict_standardized.csv


In [155]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
import os

# --- CONFIG ---
TRAIN_PATH = "/Users/jaygamage/Downloads/feature_outputs/train_featured.csv"
PREDICT_PATH = "/Users/jaygamage/Downloads/feature_outputs/predict_featured.csv"
TRAIN_OUTPUT = "/Users/jaygamage/Downloads/feature_outputs_new/2t.csv"
PREDICT_OUTPUT = "/Users/jaygamage/Downloads/feature_outputs_new/2p.csv"

# --- Column Lists ---
train_standardize_cols = [
    'advertising_promotion', 'petrol95', 'petrol92', 'auto_diesel', 'super_diesel',
    'gdp', 'ncpi', 'crudeoil', 'traffic_index'
] + [f'sales_quantity_lag_{i}' for i in range(1, 13)] + [
    'sales_quantity_rollmean_3', 'sales_quantity_rollmean_6',
    'sales_quantity_rollmean_9', 'sales_quantity_rollmean_12'
]

predict_standardize_cols = [
    "advertising_promotion",
    "gdp", "ncpi", "crudeoil", "traffic_index",
    "petrol95", "petrol92", "auto_diesel", "super_diesel"
]

# --- Load datasets ---
train_df = pd.read_csv(TRAIN_PATH, parse_dates=["date"])
predict_df = pd.read_csv(PREDICT_PATH, parse_dates=["date"])

# --- Store scalers separately ---
train_scalers = {}
predict_scalers = {}

# --- Group-wise Training Standardization ---
for group in train_df["group_code"].unique():
    group_mask = train_df["group_code"] == group
    # Train scaler for full training features
    scaler_train = StandardScaler()
    train_df.loc[group_mask, train_standardize_cols] = scaler_train.fit_transform(
        train_df.loc[group_mask, train_standardize_cols]
    )
    train_scalers[group] = scaler_train

    # Train separate scaler only for prediction feature subset
    scaler_predict = StandardScaler()
    # Use same columns but from training data; rename to match prediction naming
    predict_fit_data = train_df.loc[group_mask, [
        'advertising_promotion', 'gdp', 'ncpi', 'crudeoil', 'traffic_index',
        'petrol95', 'petrol92', 'auto_diesel', 'super_diesel'
    ]].copy()
    predict_fit_data.columns = predict_standardize_cols  # Match prediction column names
    scaler_predict.fit(predict_fit_data)
    predict_scalers[group] = scaler_predict

# --- Ensure output directory exists ---
os.makedirs(os.path.dirname(TRAIN_OUTPUT), exist_ok=True)

# --- Save standardized training data ---
train_df.to_csv(TRAIN_OUTPUT, index=False)
print(f"✅ Group-standardized training data saved to: {TRAIN_OUTPUT}")

# --- Apply matching scaler to prediction data ---
for group in predict_df["group_code"].unique():
    if group in predict_scalers:
        group_mask = predict_df["group_code"] == group
        scaler = predict_scalers[group]
        predict_df.loc[group_mask, predict_standardize_cols] = scaler.transform(
            predict_df.loc[group_mask, predict_standardize_cols]
        )
    else:
        print(f"⚠️ Warning: No scaler found for group {group}. Skipping.")

# --- Save standardized prediction data ---
predict_df.to_csv(PREDICT_OUTPUT, index=False)
print(f"✅ Prediction data standardized using matching scalers saved to: {PREDICT_OUTPUT}")

✅ Group-standardized training data saved to: /Users/jaygamage/Downloads/feature_outputs_new/2t.csv
✅ Prediction data standardized using matching scalers saved to: /Users/jaygamage/Downloads/feature_outputs_new/2p.csv


In [ ]:
#r1.4#
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import optuna
import joblib
from catboost import CatBoostRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# -------- CONFIG --------
TRAIN_PATH = "/Users/jaygamage/Downloads/feature_outputs/train_featured.csv"
PREDICT_PATH = "/Users/jaygamage/Downloads/feature_outputs/predict_featured.csv"
OUTPUT_DIR = "/Users/jaygamage/Downloads/r1.4"
TARGET = "sales_quantity"
TEST_YEAR = 2024
N_TRIALS = 100
CAT_FEATURES = ["group_code"]
LAG_COLS = ["lag_1", "lag_2", "lag_3"]
ROLLING_COLS = ["rolling_mean_3", "rolling_mean_6"]

# -------- METRICS --------
def get_metrics(y_true, y_pred):
    mae = mean_absolute_error(y_true, y_pred)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    mape = np.mean(np.abs((y_true - y_pred) / (y_true + 1e-6))) * 100
    r2 = r2_score(y_true, y_pred)
    return {"MAE": mae, "RMSE": rmse, "MAPE": mape, "R2": r2}

def plot_forecast(dates, y_true, y_pred, group_code, save_path):
    plt.figure(figsize=(10, 5))
    plt.plot(dates, y_true, label="Actual", marker="o")
    plt.plot(dates, y_pred, label="Predicted", marker="x")
    plt.title(f"CatBoost Forecast for Group {group_code}")
    plt.xlabel("Date")
    plt.ylabel("Sales Quantity")
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.savefig(save_path)
    plt.close()

# -------- FEATURE UPDATES --------
def update_recursive_features(df):
    df = df.sort_values("date")
    for lag in [1, 2, 3]:
        df[f"lag_{lag}"] = df[TARGET].shift(lag)
    for window in [3, 6]:
        df[f"rolling_mean_{window}"] = df[TARGET].rolling(window).mean()
    return df

# -------- OPTUNA OBJECTIVE --------
def build_objective_expanding(train_df, features, cat_features):
    def objective(trial):
        params = {
            "iterations": trial.suggest_int("iterations", 300, 1500),
            "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3, log=True),
            "depth": trial.suggest_int("depth", 4, 10),
            "l2_leaf_reg": trial.suggest_float("l2_leaf_reg", 1.0, 10.0),
            "bagging_temperature": trial.suggest_float("bagging_temperature", 0.0, 1.0),
            "random_strength": trial.suggest_float("random_strength", 1e-9, 10.0, log=True),
            "loss_function": "RMSE",
            "random_seed": 42,
            "verbose": 0
        }
        alpha, beta = 1000, 1000  # Penalty weights
        losses = []

        # Expanding Window Validation
        for train_end, val_year in zip([2022, 2023], [2023, 2024]):
            train_cut = train_df[train_df["date"].dt.year <= train_end]
            val_cut = train_df[train_df["date"].dt.year == val_year]

            X_train, y_train = train_cut[features], train_cut[TARGET]
            X_val, y_val = val_cut[features], val_cut[TARGET]

            model = CatBoostRegressor(**params)
            model.fit(
                X_train, y_train,
                eval_set=(X_val, y_val),
                cat_features=cat_features,
                early_stopping_rounds=50,
                verbose=0
            )

            preds = model.predict(X_val)
            rmse = np.sqrt(mean_squared_error(y_val, preds))
            mape = np.mean(np.abs((y_val - preds) / (y_val + 1e-6))) * 100
            r2 = r2_score(y_val, preds)

            loss = rmse + alpha * max(0, mape - 15) + beta * max(0.5 - r2, 0)
            losses.append(loss)

        return np.mean(losses)
    return objective

# -------- MAIN --------
def main():
    os.makedirs(f"{OUTPUT_DIR}/models", exist_ok=True)
    os.makedirs(f"{OUTPUT_DIR}/forecasts", exist_ok=True)
    os.makedirs(f"{OUTPUT_DIR}/plots", exist_ok=True)

    train_df = pd.read_csv(TRAIN_PATH, parse_dates=["date"])
    predict_df = pd.read_csv(PREDICT_PATH, parse_dates=["date"])

    group_codes = train_df["group_code"].unique()
    metric_records = []
    optuna_param_records = []

    for group in group_codes:
        print(f"🔍 Tuning group: {group}")
        train_g = train_df[train_df["group_code"] == group].copy()
        pred_g = predict_df[predict_df["group_code"] == group].copy()

        if pred_g.empty:
            print(f"⚠️ Skipping group {group} — no 2025 data.")
            continue

        drop_cols = ["date", "sales_quantity", "material_group"]
        features = [col for col in train_g.columns if col not in drop_cols and col in pred_g.columns]

        # -------- OPTUNA TUNING --------
        study = optuna.create_study(direction="minimize")
        study.optimize(build_objective_expanding(train_g, features, CAT_FEATURES), n_trials=N_TRIALS)
        best_params = study.best_params
        best_params.update({"loss_function": "RMSE", "random_seed": 42, "verbose": 0})
        optuna_param_records.append({**best_params, "group_code": group})

        # -------- FINAL MODEL TRAINING --------
        final_train = train_g[train_g["date"].dt.year <= TEST_YEAR]
        X_train, y_train = final_train[features], final_train[TARGET]

        model = CatBoostRegressor(**best_params)
        model.fit(X_train, y_train, cat_features=CAT_FEATURES)
        joblib.dump(model, f"{OUTPUT_DIR}/models/{group}_catboost_optuna.pkl")

        # -------- VALIDATION --------
        val_cut = train_g[train_g["date"].dt.year == TEST_YEAR]
        val_preds = model.predict(val_cut[features])
        val_cut["prediction"] = val_preds
        metrics = get_metrics(val_cut[TARGET], val_preds)
        metrics["group_code"] = group
        metric_records.append(metrics)
        plot_forecast(val_cut["date"], val_cut[TARGET], val_preds, group, f"{OUTPUT_DIR}/plots/{group}_forecast.png")

        # -------- RECURSIVE FORECASTING --------
        history = pd.concat([train_g, pred_g], ignore_index=True).sort_values("date")
        for idx, row in pred_g.iterrows():
            current_date = row["date"]
            temp_history = history[history["date"] <= current_date].copy()
            temp_history = update_recursive_features(temp_history)
            updated_row = temp_history[temp_history["date"] == current_date]
            pred_features = updated_row[features]
            pred_value = model.predict(pred_features)[0]
            history.loc[history["date"] == current_date, TARGET] = pred_value

        # Save Recursive Forecast
        pred_g = history[history["date"].dt.year == 2025][["date", "group_code", TARGET]]
        pred_g.rename(columns={TARGET: "predicted_sales_quantity"}, inplace=True)
        pred_g.to_csv(f"{OUTPUT_DIR}/forecasts/{group}_forecast.csv", index=False)
        print(f"📈 Group {group} complete.")

    pd.DataFrame(metric_records).to_csv(f"{OUTPUT_DIR}/metrics.csv", index=False)
    pd.DataFrame(optuna_param_records).to_csv(f"{OUTPUT_DIR}/optuna_params.csv", index=False)
    print("✅ All groups trained, validated, and recursively forecasted.")

if __name__ == "__main__":
    main()

[I 2025-04-26 01:34:49,835] A new study created in memory with name: no-name-ded43287-ec25-45bf-9ef2-15ad3fa0bfee
[I 2025-04-26 01:34:49,970] Trial 0 finished with value: 7862.8278905478855 and parameters: {'iterations': 1442, 'learning_rate': 0.059377922994710765, 'depth': 4, 'l2_leaf_reg': 5.94584466141386, 'bagging_temperature': 0.5601012068916709, 'random_strength': 0.0009868037116197925}. Best is trial 0 with value: 7862.8278905478855.


🔍 Tuning group: 0


[I 2025-04-26 01:34:51,390] Trial 1 finished with value: 13324.817074049908 and parameters: {'iterations': 929, 'learning_rate': 0.07468696243592728, 'depth': 10, 'l2_leaf_reg': 5.6575516192653055, 'bagging_temperature': 0.04816369012186028, 'random_strength': 3.979801658079165e-05}. Best is trial 0 with value: 7862.8278905478855.
[I 2025-04-26 01:34:51,770] Trial 2 finished with value: 12163.14090939039 and parameters: {'iterations': 1145, 'learning_rate': 0.17505703867986538, 'depth': 9, 'l2_leaf_reg': 3.5217401211785617, 'bagging_temperature': 0.045071577450199074, 'random_strength': 1.693722566177437e-08}. Best is trial 0 with value: 7862.8278905478855.
[I 2025-04-26 01:34:52,186] Trial 3 finished with value: 11206.47809005453 and parameters: {'iterations': 359, 'learning_rate': 0.16500946102720374, 'depth': 9, 'l2_leaf_reg': 4.862791911475443, 'bagging_temperature': 0.8482046762848681, 'random_strength': 0.02706531472333222}. Best is trial 0 with value: 7862.8278905478855.
[I 2025

📈 Group 0 complete.
🔍 Tuning group: 1


[I 2025-04-26 01:35:11,953] Trial 0 finished with value: 16522.76897660445 and parameters: {'iterations': 375, 'learning_rate': 0.24043129702836444, 'depth': 9, 'l2_leaf_reg': 7.864814763915595, 'bagging_temperature': 0.06265824741161707, 'random_strength': 0.07868469694618}. Best is trial 0 with value: 16522.76897660445.
[I 2025-04-26 01:35:12,346] Trial 1 finished with value: 17080.264207962267 and parameters: {'iterations': 497, 'learning_rate': 0.015272028513940482, 'depth': 8, 'l2_leaf_reg': 5.545087406519544, 'bagging_temperature': 0.24651714767225708, 'random_strength': 0.0002244802310683211}. Best is trial 0 with value: 16522.76897660445.
[I 2025-04-26 01:35:12,635] Trial 2 finished with value: 17798.94546214555 and parameters: {'iterations': 585, 'learning_rate': 0.18486443571803496, 'depth': 9, 'l2_leaf_reg': 4.091245121521282, 'bagging_temperature': 0.3419030223758903, 'random_strength': 0.0009576866433728101}. Best is trial 0 with value: 16522.76897660445.
[I 2025-04-26 01:

...

📈 Group 20 complete.
🔍 Tuning group: 21


[I 2025-04-26 01:44:55,999] Trial 0 finished with value: 8923.843117420502 and parameters: {'iterations': 674, 'learning_rate': 0.10377507181851833, 'depth': 10, 'l2_leaf_reg': 7.380739782754019, 'bagging_temperature': 0.1198990654155353, 'random_strength': 0.0011325289454377453}. Best is trial 0 with value: 8923.843117420502.
[I 2025-04-26 01:44:56,287] Trial 1 finished with value: 11072.155682739889 and parameters: {'iterations': 1255, 'learning_rate': 0.031085240370832462, 'depth': 8, 'l2_leaf_reg': 7.011734877373566, 'bagging_temperature': 0.6865736905260222, 'random_strength': 0.008796385749954087}. Best is trial 0 with value: 8923.843117420502.
[I 2025-04-26 01:44:56,477] Trial 2 finished with value: 12677.455486948835 and parameters: {'iterations': 922, 'learning_rate': 0.04813817099230789, 'depth': 9, 'l2_leaf_reg': 1.9323242057740078, 'bagging_temperature': 0.8046432276872053, 'random_strength': 2.48929302502421e-08}. Best is trial 0 with value: 8923.843117420502.
[I 2025-04-2

📈 Group 21 complete.
🔍 Tuning group: 22


[I 2025-04-26 01:45:09,817] Trial 1 finished with value: 13272.530668130488 and parameters: {'iterations': 762, 'learning_rate': 0.011128484710845859, 'depth': 7, 'l2_leaf_reg': 9.43248517239923, 'bagging_temperature': 0.43223991384172056, 'random_strength': 3.6674967872617937e-07}. Best is trial 0 with value: 10383.113636909187.
[I 2025-04-26 01:45:09,953] Trial 2 finished with value: 10596.32913519975 and parameters: {'iterations': 636, 'learning_rate': 0.016070392993374392, 'depth': 4, 'l2_leaf_reg': 9.316728652297195, 'bagging_temperature': 0.4983602268429833, 'random_strength': 1.3645697987686438e-07}. Best is trial 0 with value: 10383.113636909187.
[I 2025-04-26 01:45:09,986] Trial 3 finished with value: 10398.416387906396 and parameters: {'iterations': 1130, 'learning_rate': 0.1983586688958182, 'depth': 4, 'l2_leaf_reg': 8.201486294756217, 'bagging_temperature': 0.45467654471623387, 'random_strength': 2.1219332490602096}. Best is trial 0 with value: 10383.113636909187.
[I 2025-0

📈 Group 22 complete.
✅ All groups trained, validated, and recursively forecasted.


In [16]:
# r1.4 — CatBoost Forecasting with log1p/expm1
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import optuna
import joblib
from catboost import CatBoostRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# -------- CONFIG --------
TRAIN_PATH = "/Users/jaygamage/Downloads/feature_outputs/train_featured.csv"
PREDICT_PATH = "/Users/jaygamage/Downloads/feature_outputs/predict_featured.csv"
OUTPUT_DIR = "/Users/jaygamage/Downloads/r1.4.log"
TARGET = "sales_quantity"
TEST_YEAR = 2024
N_TRIALS = 100
CAT_FEATURES = ["group_code"]
LAG_COLS = ["lag_1", "lag_2", "lag_3"]
ROLLING_COLS = ["rolling_mean_3", "rolling_mean_6"]

# -------- METRICS --------
def get_metrics(y_true, y_pred):
    y_true, y_pred = np.expm1(y_true), np.expm1(y_pred)
    mae = mean_absolute_error(y_true, y_pred)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    mape = np.mean(np.abs((y_true - y_pred) / (y_true + 1e-6))) * 100
    r2 = r2_score(y_true, y_pred)
    return {"MAE": mae, "RMSE": rmse, "MAPE": mape, "R2": r2}

def plot_forecast(dates, y_true, y_pred, group_code, save_path):
    y_true, y_pred = np.expm1(y_true), np.expm1(y_pred)
    plt.figure(figsize=(10, 5))
    plt.plot(dates, y_true, label="Actual", marker="o")
    plt.plot(dates, y_pred, label="Predicted", marker="x")
    plt.title(f"CatBoost Forecast for Group {group_code}")
    plt.xlabel("Date")
    plt.ylabel("Sales Quantity")
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.savefig(save_path)
    plt.close()

# -------- FEATURE UPDATES --------
def update_recursive_features(df):
    df = df.sort_values("date")
    for lag in [1, 2, 3]:
        df[f"lag_{lag}"] = df[TARGET].shift(lag)
    for window in [3, 6]:
        df[f"rolling_mean_{window}"] = df[TARGET].rolling(window).mean()
    return df

# -------- OPTUNA OBJECTIVE --------
def build_objective_expanding(train_df, features, cat_features):
    def objective(trial):
        params = {
            "iterations": trial.suggest_int("iterations", 300, 1500),
            "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3, log=True),
            "depth": trial.suggest_int("depth", 4, 10),
            "l2_leaf_reg": trial.suggest_float("l2_leaf_reg", 1.0, 10.0),
            "bagging_temperature": trial.suggest_float("bagging_temperature", 0.0, 1.0),
            "random_strength": trial.suggest_float("random_strength", 1e-9, 10.0, log=True),
            "loss_function": "RMSE",
            "random_seed": 42,
            "verbose": 0
        }
        alpha, beta = 1000, 1000
        losses = []

        for train_end, val_year in zip([2022, 2023], [2023, 2024]):
            train_cut = train_df[train_df["date"].dt.year <= train_end]
            val_cut = train_df[train_df["date"].dt.year == val_year]

            X_train, y_train = train_cut[features], np.log1p(train_cut[TARGET])
            X_val, y_val = val_cut[features], np.log1p(val_cut[TARGET])

            model = CatBoostRegressor(**params)
            model.fit(
                X_train, y_train,
                eval_set=(X_val, y_val),
                cat_features=cat_features,
                early_stopping_rounds=50,
                verbose=0
            )

            preds = model.predict(X_val)
            rmse = np.sqrt(mean_squared_error(y_val, preds))
            mape = np.mean(np.abs((np.expm1(y_val) - np.expm1(preds)) / (np.expm1(y_val) + 1e-6))) * 100
            r2 = r2_score(np.expm1(y_val), np.expm1(preds))

            loss = rmse + alpha * max(0, mape - 15) + beta * max(0.5 - r2, 0)
            losses.append(loss)

        return np.mean(losses)
    return objective

# -------- MAIN --------
def main():
    os.makedirs(f"{OUTPUT_DIR}/models", exist_ok=True)
    os.makedirs(f"{OUTPUT_DIR}/forecasts", exist_ok=True)
    os.makedirs(f"{OUTPUT_DIR}/plots", exist_ok=True)

    train_df = pd.read_csv(TRAIN_PATH, parse_dates=["date"])
    predict_df = pd.read_csv(PREDICT_PATH, parse_dates=["date"])

    group_codes = train_df["group_code"].unique()
    metric_records = []
    optuna_param_records = []

    for group in group_codes:
        print(f"🔍 Tuning group: {group}")
        train_g = train_df[train_df["group_code"] == group].copy()
        pred_g = predict_df[predict_df["group_code"] == group].copy()

        if pred_g.empty:
            print(f"⚠️ Skipping group {group} — no 2025 data.")
            continue

        drop_cols = ["date", "sales_quantity", "material_group"]
        features = [col for col in train_g.columns if col not in drop_cols and col in pred_g.columns]

        # -------- OPTUNA TUNING --------
        study = optuna.create_study(direction="minimize")
        study.optimize(build_objective_expanding(train_g, features, CAT_FEATURES), n_trials=N_TRIALS)
        best_params = study.best_params
        best_params.update({"loss_function": "RMSE", "random_seed": 42, "verbose": 0})
        optuna_param_records.append({**best_params, "group_code": group})

        # -------- FINAL MODEL TRAINING --------
        final_train = train_g[train_g["date"].dt.year <= TEST_YEAR]
        X_train, y_train = final_train[features], np.log1p(final_train[TARGET])

        model = CatBoostRegressor(**best_params)
        model.fit(X_train, y_train, cat_features=CAT_FEATURES)
        joblib.dump(model, f"{OUTPUT_DIR}/models/{group}_catboost_optuna.pkl")

        # -------- VALIDATION --------
        val_cut = train_g[train_g["date"].dt.year == TEST_YEAR]
        val_preds = model.predict(val_cut[features])
        val_cut["prediction"] = val_preds
        metrics = get_metrics(np.log1p(val_cut[TARGET]), val_preds)
        metrics["group_code"] = group
        metric_records.append(metrics)
        plot_forecast(val_cut["date"], np.log1p(val_cut[TARGET]), val_preds, group, f"{OUTPUT_DIR}/plots/{group}_forecast.png")

        # -------- RECURSIVE FORECASTING --------
        history = pd.concat([train_g, pred_g], ignore_index=True).sort_values("date")
        for idx, row in pred_g.iterrows():
            current_date = row["date"]
            temp_history = history[history["date"] <= current_date].copy()
            temp_history = update_recursive_features(temp_history)
            updated_row = temp_history[temp_history["date"] == current_date]
            pred_features = updated_row[features]
            pred_value = model.predict(pred_features)[0]
            history.loc[history["date"] == current_date, TARGET] = np.expm1(pred_value)

        # Save Recursive Forecast
        pred_g = history[history["date"].dt.year == 2025][["date", "group_code", TARGET]]
        pred_g.rename(columns={TARGET: "predicted_sales_quantity"}, inplace=True)
        pred_g.to_csv(f"{OUTPUT_DIR}/forecasts/{group}_forecast.csv", index=False)
        print(f"📈 Group {group} complete.")

    pd.DataFrame(metric_records).to_csv(f"{OUTPUT_DIR}/metrics.csv", index=False)
    pd.DataFrame(optuna_param_records).to_csv(f"{OUTPUT_DIR}/optuna_params.csv", index=False)
    print("✅ All groups trained, validated, and recursively forecasted.")

if __name__ == "__main__":
    main()

[I 2025-05-03 00:40:01,740] A new study created in memory with name: no-name-ecb70d49-8f83-41c0-b84c-aa9438352871


🔍 Tuning group: 0


[I 2025-05-03 00:40:02,015] Trial 0 finished with value: 3165.9760354382156 and parameters: {'iterations': 1393, 'learning_rate': 0.20196179893844332, 'depth': 7, 'l2_leaf_reg': 3.214333787198278, 'bagging_temperature': 0.590304467571771, 'random_strength': 0.0003109714169900954}. Best is trial 0 with value: 3165.9760354382156.
[I 2025-05-03 00:40:02,875] Trial 1 finished with value: 9729.199524857988 and parameters: {'iterations': 342, 'learning_rate': 0.05549063656973899, 'depth': 10, 'l2_leaf_reg': 2.098848572041677, 'bagging_temperature': 0.6450052761523128, 'random_strength': 0.0011529426085971708}. Best is trial 0 with value: 3165.9760354382156.
[I 2025-05-03 00:40:02,961] Trial 2 finished with value: 2123.4184023857606 and parameters: {'iterations': 1148, 'learning_rate': 0.06624827491674914, 'depth': 4, 'l2_leaf_reg': 5.0997608373468974, 'bagging_temperature': 0.8154930069369084, 'random_strength': 3.653963796542995e-06}. Best is trial 2 with value: 2123.4184023857606.
[I 2025-

📈 Group 0 complete.
🔍 Tuning group: 1


[I 2025-05-03 00:40:26,217] Trial 0 finished with value: 8343.069456997522 and parameters: {'iterations': 936, 'learning_rate': 0.03346584400853696, 'depth': 7, 'l2_leaf_reg': 7.3009825140010856, 'bagging_temperature': 0.08111120045742115, 'random_strength': 7.370233846432109e-07}. Best is trial 0 with value: 8343.069456997522.
[I 2025-05-03 00:40:26,454] Trial 1 finished with value: 7408.245485128766 and parameters: {'iterations': 1357, 'learning_rate': 0.05730612663276169, 'depth': 7, 'l2_leaf_reg': 7.077684544732586, 'bagging_temperature': 0.82988973023215, 'random_strength': 3.276800698970223e-05}. Best is trial 1 with value: 7408.245485128766.
[I 2025-05-03 00:40:26,801] Trial 2 finished with value: 6480.468096438385 and parameters: {'iterations': 782, 'learning_rate': 0.010475552508060015, 'depth': 6, 'l2_leaf_reg': 9.80855460198569, 'bagging_temperature': 0.41351010505101926, 'random_strength': 2.385443917848389}. Best is trial 2 with value: 6480.468096438385.
[I 2025-05-03 00:4

...

📈 Group 20 complete.
🔍 Tuning group: 21


[I 2025-05-03 00:47:22,064] Trial 0 finished with value: 4463.962660009485 and parameters: {'iterations': 665, 'learning_rate': 0.012585927194192569, 'depth': 8, 'l2_leaf_reg': 9.127472838637079, 'bagging_temperature': 0.4039999968454916, 'random_strength': 0.028678358904085495}. Best is trial 0 with value: 4463.962660009485.
[I 2025-05-03 00:47:23,542] Trial 1 finished with value: 5767.358534874508 and parameters: {'iterations': 426, 'learning_rate': 0.029949261873055848, 'depth': 10, 'l2_leaf_reg': 3.555489214131275, 'bagging_temperature': 0.5826916400275933, 'random_strength': 0.00010848398097764877}. Best is trial 0 with value: 4463.962660009485.
[I 2025-05-03 00:47:24,689] Trial 2 finished with value: 6497.671861556337 and parameters: {'iterations': 379, 'learning_rate': 0.13122369324346764, 'depth': 10, 'l2_leaf_reg': 9.086905908041151, 'bagging_temperature': 0.07655168717184346, 'random_strength': 5.215535910374168e-07}. Best is trial 0 with value: 4463.962660009485.
[I 2025-05-

📈 Group 21 complete.
🔍 Tuning group: 22


[I 2025-05-03 00:47:35,353] Trial 0 finished with value: 5933.9245625253825 and parameters: {'iterations': 1217, 'learning_rate': 0.010249019988928292, 'depth': 6, 'l2_leaf_reg': 8.220172277002073, 'bagging_temperature': 0.9640020784820162, 'random_strength': 0.0007386484219598649}. Best is trial 0 with value: 5933.9245625253825.
[I 2025-05-03 00:47:36,756] Trial 1 finished with value: 8151.685563833528 and parameters: {'iterations': 554, 'learning_rate': 0.034429194956899105, 'depth': 10, 'l2_leaf_reg': 6.00875755648834, 'bagging_temperature': 0.6158106297867941, 'random_strength': 1.1006042780010554e-07}. Best is trial 0 with value: 5933.9245625253825.
[I 2025-05-03 00:47:36,871] Trial 2 finished with value: 4020.0886576951025 and parameters: {'iterations': 412, 'learning_rate': 0.015371006835710513, 'depth': 4, 'l2_leaf_reg': 1.4298804574960244, 'bagging_temperature': 0.4998470372871674, 'random_strength': 5.229569068426381e-07}. Best is trial 2 with value: 4020.0886576951025.
[I 20

📈 Group 22 complete.
✅ All groups trained, validated, and recursively forecasted.


In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import optuna
import joblib
from catboost import CatBoostRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# -------- CONFIG --------
TRAIN_PATH = "/Users/jaygamage/Downloads/feature_outputs/train_featured.csv"
PREDICT_PATH = "/Users/jaygamage/Downloads/feature_outputs/predict_featured.csv"
OUTPUT_DIR = "/Users/jaygamage/Downloads/r1.4lg_refined"
TARGET = "sales_quantity"
TEST_YEAR = 2024
N_TRIALS = 100
CAT_FEATURES = ["group_code"]

# -------- METRICS --------
def get_metrics(y_true, y_pred):
    y_true, y_pred = np.expm1(y_true), np.expm1(y_pred)
    mae = mean_absolute_error(y_true, y_pred)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    mape = np.mean(np.abs((y_true - y_pred) / (y_true + 1e-6))) * 100
    r2 = r2_score(y_true, y_pred)
    return {"MAE": mae, "RMSE": rmse, "MAPE": mape, "R2": r2}

def plot_forecast(dates, y_true, y_pred, group_code, save_path):
    y_true, y_pred = np.expm1(y_true), np.expm1(y_pred)
    plt.figure(figsize=(10, 5))
    plt.plot(dates, y_true, label="Actual", marker="o")
    plt.plot(dates, y_pred, label="Predicted", marker="x")
    plt.title(f"Forecast for Group {group_code}")
    plt.xlabel("Date")
    plt.ylabel("Sales Quantity")
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.savefig(save_path)
    plt.close()

# -------- FEATURE UPDATER --------
def update_recursive_features(df):
    df = df.sort_values("date")
    for lag in [1, 2, 3]:
        df[f"lag_{lag}"] = df[TARGET].shift(lag)
    for window in [3, 6]:
        df[f"rolling_mean_{window}"] = df[TARGET].rolling(window).mean()
    return df

# -------- OPTUNA OBJECTIVE --------
def build_objective(train_df, features, cat_features):
    def objective(trial):
        params = {
            "iterations": trial.suggest_int("iterations", 300, 1500),
            "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3, log=True),
            "depth": trial.suggest_int("depth", 4, 10),
            "l2_leaf_reg": trial.suggest_float("l2_leaf_reg", 1.0, 10.0),
            "bagging_temperature": trial.suggest_float("bagging_temperature", 0.0, 1.0),
            "random_strength": trial.suggest_float("random_strength", 1e-9, 10.0, log=True),
            "loss_function": "RMSE", "random_seed": 42, "verbose": 0
        }
        alpha, beta = 1000, 1000
        losses = []

        for train_end, val_year in zip([2022, 2023], [2023, 2024]):
            train_cut = train_df[train_df["date"].dt.year <= train_end]
            val_cut = train_df[train_df["date"].dt.year == val_year]

            X_train, y_train = train_cut[features], np.log1p(train_cut[TARGET])
            X_val, y_val = val_cut[features], np.log1p(val_cut[TARGET])

            model = CatBoostRegressor(**params)
            model.fit(X_train, y_train, eval_set=(X_val, y_val), cat_features=cat_features, early_stopping_rounds=50, verbose=0)

            preds = model.predict(X_val)
            rmse = np.sqrt(mean_squared_error(y_val, preds))
            mape = np.mean(np.abs((np.expm1(y_val) - np.expm1(preds)) / (np.expm1(y_val) + 1e-6))) * 100
            r2 = r2_score(np.expm1(y_val), np.expm1(preds))

            loss = rmse + alpha * max(0, mape - 15) + beta * max(0.5 - r2, 0)
            losses.append(loss)

        return np.mean(losses)
    return objective

# -------- MAIN --------
def main():
    os.makedirs(f"{OUTPUT_DIR}/models", exist_ok=True)
    os.makedirs(f"{OUTPUT_DIR}/forecasts", exist_ok=True)
    os.makedirs(f"{OUTPUT_DIR}/plots", exist_ok=True)

    train_df = pd.read_csv(TRAIN_PATH, parse_dates=["date"])
    predict_df = pd.read_csv(PREDICT_PATH, parse_dates=["date"])
    group_codes = train_df["group_code"].unique()

    metric_records = []
    train_metrics = []
    optuna_param_records = []

    for group in group_codes:
        print(f"🔍 Tuning group: {group}")
        train_g = train_df[train_df["group_code"] == group].copy()
        pred_g = predict_df[predict_df["group_code"] == group].copy()

        if pred_g.empty:
            print(f"⚠️ Skipping group {group} — no 2025 data.")
            continue

        drop_cols = ["date", "sales_quantity", "material_group"]
        features = [col for col in train_g.columns if col not in drop_cols and col in pred_g.columns]

        # -------- TUNING --------
        study = optuna.create_study(direction="minimize")
        study.optimize(build_objective(train_g, features, CAT_FEATURES), n_trials=N_TRIALS)
        best_params = study.best_params
        best_params.update({"loss_function": "RMSE", "random_seed": 42, "verbose": 0})
        optuna_param_records.append({**best_params, "group_code": group})

        # -------- FINAL TRAINING --------
        final_train = train_g[train_g["date"].dt.year <= TEST_YEAR]
        X_train, y_train = final_train[features], np.log1p(final_train[TARGET])
        model = CatBoostRegressor(**best_params)
        model.fit(X_train, y_train, cat_features=CAT_FEATURES)
        joblib.dump(model, f"{OUTPUT_DIR}/models/{group}_catboost_optuna.pkl")

        # -------- VALIDATION --------
        val_cut = train_g[train_g["date"].dt.year == TEST_YEAR]
        val_preds = model.predict(val_cut[features])
        val_cut["prediction"] = val_preds
        metrics = get_metrics(np.log1p(val_cut[TARGET]), val_preds)
        metrics["group_code"] = group
        metric_records.append(metrics)
        plot_forecast(val_cut["date"], np.log1p(val_cut[TARGET]), val_preds, group, f"{OUTPUT_DIR}/plots/{group}_forecast.png")

        # -------- TRAIN METRICS --------
        train_preds = model.predict(X_train)
        train_m = get_metrics(y_train, train_preds)
        train_m["group_code"] = group
        train_metrics.append(train_m)

        # -------- RECURSIVE FORECAST --------
        history = pd.concat([train_g, pred_g], ignore_index=True).sort_values("date")
        for idx, row in pred_g.iterrows():
            current_date = row["date"]
            temp_history = history[history["date"] <= current_date].copy()
            temp_history = update_recursive_features(temp_history)
            updated_row = temp_history[temp_history["date"] == current_date]
            pred_features = updated_row[features]
            pred_value = model.predict(pred_features)[0]
            history.loc[history["date"] == current_date, TARGET] = np.expm1(pred_value)

        pred_g = history[history["date"].dt.year == 2025][["date", "group_code", TARGET]]
        pred_g.rename(columns={TARGET: "predicted_sales_quantity"}, inplace=True)
        pred_g.to_csv(f"{OUTPUT_DIR}/forecasts/{group}_forecast.csv", index=False)

        print(f"✅ Group {group} complete.")

    pd.DataFrame(metric_records).to_csv(f"{OUTPUT_DIR}/cv_metrics.csv", index=False)
    pd.DataFrame(train_metrics).to_csv(f"{OUTPUT_DIR}/train_metrics.csv", index=False)
    pd.DataFrame(optuna_param_records).to_csv(f"{OUTPUT_DIR}/optuna_params.csv", index=False)
    print("🎯 Forecasting complete for all groups.")

if __name__ == "__main__":
    main()

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[I 2025-05-18 22:22:50,015] A new study created in memory with name: no-name-52939b6e-ff2f-4d8e-a1d3-bfdd9b565e2f


🔍 Tuning group: 0


[I 2025-05-18 22:22:53,234] Trial 0 finished with value: 7073.910151060184 and parameters: {'iterations': 1354, 'learning_rate': 0.025484256805413694, 'depth': 10, 'l2_leaf_reg': 9.467786197997103, 'bagging_temperature': 0.6300968763906004, 'random_strength': 1.4443065366189583e-07}. Best is trial 0 with value: 7073.910151060184.
[I 2025-05-18 22:22:53,700] Trial 1 finished with value: 8786.047324959485 and parameters: {'iterations': 1440, 'learning_rate': 0.2009301000902353, 'depth': 10, 'l2_leaf_reg': 5.893658480207052, 'bagging_temperature': 0.3783147594794437, 'random_strength': 9.052905700940828e-07}. Best is trial 0 with value: 7073.910151060184.
[I 2025-05-18 22:22:53,883] Trial 2 finished with value: 3383.7642798355714 and parameters: {'iterations': 517, 'learning_rate': 0.03170550622451818, 'depth': 6, 'l2_leaf_reg': 4.6884116650815235, 'bagging_temperature': 0.38444690960009253, 'random_strength': 0.0006034664776730023}. Best is trial 2 with value: 3383.7642798355714.
[I 2025

✅ Group 0 complete.
🔍 Tuning group: 1


[I 2025-05-18 22:23:09,872] Trial 0 finished with value: 11552.222287033379 and parameters: {'iterations': 556, 'learning_rate': 0.016267854746094608, 'depth': 10, 'l2_leaf_reg': 9.690565471599154, 'bagging_temperature': 0.7758897606091315, 'random_strength': 3.948066487913383e-05}. Best is trial 0 with value: 11552.222287033379.
[I 2025-05-18 22:23:10,138] Trial 1 finished with value: 6764.225551572219 and parameters: {'iterations': 444, 'learning_rate': 0.032156596899750436, 'depth': 7, 'l2_leaf_reg': 7.705268402826264, 'bagging_temperature': 0.22374663150102958, 'random_strength': 0.1541363433139112}. Best is trial 1 with value: 6764.225551572219.
[I 2025-05-18 22:23:10,967] Trial 2 finished with value: 8935.589926068822 and parameters: {'iterations': 1490, 'learning_rate': 0.04639943586624705, 'depth': 10, 'l2_leaf_reg': 2.904662120489702, 'bagging_temperature': 0.5048968415453979, 'random_strength': 0.07610116370129404}. Best is trial 1 with value: 6764.225551572219.
[I 2025-05-18

...

✅ Group 20 complete.
🔍 Tuning group: 21


[I 2025-05-18 22:29:41,574] Trial 0 finished with value: 5141.536593851581 and parameters: {'iterations': 771, 'learning_rate': 0.0178093390638589, 'depth': 7, 'l2_leaf_reg': 3.3181697668640675, 'bagging_temperature': 0.9008064813897734, 'random_strength': 0.0086214409739151}. Best is trial 0 with value: 5141.536593851581.
[I 2025-05-18 22:29:41,769] Trial 1 finished with value: 5918.434466397536 and parameters: {'iterations': 848, 'learning_rate': 0.043134040213088795, 'depth': 8, 'l2_leaf_reg': 2.267061404505846, 'bagging_temperature': 0.414853860322119, 'random_strength': 1.259585717489272e-07}. Best is trial 0 with value: 5141.536593851581.
[I 2025-05-18 22:29:42,789] Trial 2 finished with value: 5218.47050797125 and parameters: {'iterations': 508, 'learning_rate': 0.05767708085084177, 'depth': 10, 'l2_leaf_reg': 8.44374088075903, 'bagging_temperature': 0.1427699355534987, 'random_strength': 6.750608396174588e-05}. Best is trial 0 with value: 5141.536593851581.
[I 2025-05-18 22:29:

✅ Group 21 complete.
🔍 Tuning group: 22


[I 2025-05-18 22:29:59,131] Trial 0 finished with value: 6780.017100189523 and parameters: {'iterations': 1137, 'learning_rate': 0.07635683712178795, 'depth': 9, 'l2_leaf_reg': 5.853482894737766, 'bagging_temperature': 0.003305025338567913, 'random_strength': 2.3543687082571434e-08}. Best is trial 0 with value: 6780.017100189523.
[I 2025-05-18 22:29:59,426] Trial 1 finished with value: 7897.997644479645 and parameters: {'iterations': 1269, 'learning_rate': 0.06371331437107454, 'depth': 8, 'l2_leaf_reg': 7.9834731172245705, 'bagging_temperature': 0.058788231217986264, 'random_strength': 0.019656624939451982}. Best is trial 0 with value: 6780.017100189523.
[I 2025-05-18 22:29:59,645] Trial 2 finished with value: 4972.117365724986 and parameters: {'iterations': 1074, 'learning_rate': 0.035508853166872086, 'depth': 5, 'l2_leaf_reg': 9.622341374757822, 'bagging_temperature': 0.32982496662628125, 'random_strength': 5.529745377480038}. Best is trial 2 with value: 4972.117365724986.
[I 2025-05

✅ Group 22 complete.
🎯 Forecasting complete for all groups.
